## Startup

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from hmmlearn import hmm

import os
import pickle

In [2]:
random_state=42
np.random.seed(random_state)

In [3]:
from scripts.params import get_params

params = get_params()

## Data Retrieval

In [4]:
dataroute=os.path.join("..",  "data")
dumproute=os.path.join("..",  "dump")
resultsroute=os.path.join("..",  "results")

In [5]:
name=f'finaldf_train_{params["tablename"]}.pickle'
filename=os.path.join(dataroute, name)
with open(filename, 'rb') as handle:
    df=pickle.load(handle)


In [6]:
df.head()

,^MERV_rets,^MERV_log_rets,^MERV_gk_vol,GGAL.BA_rets,GGAL.BA_log_rets,GGAL.BA_gk_vol,GGAL_rets,GGAL_log_rets,GGAL_gk_vol,YPFD.BA_rets,...,BBAR.BA_gk_vol,BBAR_rets,BBAR_log_rets,BBAR_gk_vol,USD_rets,USD_log_rets,USD_gk_vol,USD_^MERV_rets,USD_^MERV_log_rets,USD_^MERV_gk_vol
2013-01-03,0.007552,0.007524,0.000129,0.010616,0.010560,0.000677,-0.012748,-0.012830,0.001228,-0.006862,...,0.000169,-0.005725,-0.005742,0.000960,0.008830,0.008792,0.000014,0.001247,0.001246,0.000129
2013-01-04,0.007092,0.007067,0.000158,-0.006303,-0.006323,0.000208,-0.010043,-0.010094,0.000554,0.004936,...,0.000406,-0.019194,-0.019381,0.000635,0.018043,0.017883,0.000133,-0.005727,-0.005744,0.000158
2013-01-07,-0.001035,-0.001035,0.000022,0.002114,0.002112,0.000063,-0.014493,-0.014599,0.000517,0.010805,...,0.000492,0.015656,0.015534,0.000511,-0.002489,-0.002492,0.000048,-0.009769,-0.009817,0.000022
2013-01-08,0.008285,0.008251,0.000082,-0.008439,-0.008475,0.000153,-0.016176,-0.016309,0.001085,0.049563,...,0.000438,-0.015414,-0.015534,0.000642,0.015356,0.015239,0.000064,-0.001117,-0.001118,0.000082
2013-01-09,0.017826,0.017669,0.000273,0.000000,0.000000,0.000000,0.011958,0.011887,0.005238,0.000000,...,0.000000,-0.003914,-0.003921,0.000147,-0.008671,-0.008709,0.001065,0.017245,0.017098,0.000273


In [7]:
tickerlist=params["tickerlist"]

## HMM Training

In [8]:
range_states=range(1,16)
emptydf=pd.DataFrame(columns=["AIC", "BIC"], index=range_states)
emptydf.fillna(np.inf, inplace=True)
results_dict_df={stock:emptydf for stock in tickerlist}

In [9]:
aic_best_model={stock:None for stock in tickerlist}
bic_best_model={stock:None for stock in tickerlist}

aic_best_residuals={stock:None for stock in tickerlist}
bic_best_residuals={stock:None for stock in tickerlist}

In [10]:
for stock in tickerlist:
    columns = [f'{stock}_log_rets', f'{stock}_gk_vol']
    insample_data = df[columns]

    param_dict={
        "covariance_type" : "diag", 
        "n_iter" : 500,
        "random_state" : random_state
        #no voy a usar startprob_prior por devlog 20-06-23
        }

    for nstate in range_states:
        print(stock, nstate)
        model = hmm.GaussianHMM(n_components= nstate, **param_dict, verbose=False)
        results = model.fit(insample_data)

        convergence=results.monitor_.converged
        # esta es la condición de si el modelo convergió
        
        all_states_found=np.isclose(a=(model.transmat_.sum(axis=1)), b=1).all()
        # esta es la condición de que todos los estados (nstates) hayan sido observados
        # si no, alguna fila en la matriz de transición del modelo son 0.
        # el errormsg es "Some rows of transmat_ have zero sum because no transition from the state was ever observed".

        if convergence and all_states_found:
            try:
                results_dict_df[stock].loc[nstate, "AIC"]=model.aic(insample_data)
                results_dict_df[stock].loc[nstate, "BIC"]=model.bic(insample_data)
            except ValueError:
                pass
        else: 
            print(">"*10,f"{stock} {nstate} did not converge")
            results_dict_df[stock].loc[nstate, "BIC"]=np.inf
            results_dict_df[stock].loc[nstate, "BIC"]=np.inf

^MERV 1


Model is not converging.  Current: 18854.96287782649 is not greater than 18856.523959937764. Delta is -1.5610821112750273


^MERV 2
^MERV 3


Model is not converging.  Current: 18922.516761095125 is not greater than 18922.640342060477. Delta is -0.1235809653517208
Model is not converging.  Current: 18926.94497080564 is not greater than 18929.056224078548. Delta is -2.111253272905742


^MERV 4
^MERV 5
^MERV 6
^MERV 7


...

Model is not converging.  Current: 16596.70708316306 is not greater than 16598.90454582374. Delta is -2.197462660682504


BBAR 14


Model is not converging.  Current: 17104.79696368527 is not greater than 17105.61871082823. Delta is -0.8217471429597936


BBAR 15


Model is not converging.  Current: 17121.986346512214 is not greater than 17128.800132663815. Delta is -6.813786151600652


In [12]:
for stock in tickerlist:
    columns = [f'{stock}_log_rets', f'{stock}_gk_vol']
    insample_data = df[columns]
    
    best_aic_nstate=results_dict_df[stock]["AIC"].astype(float).idxmin()
    best_bic_nstate=results_dict_df[stock]["BIC"].astype(float).idxmin()
    print(f"For stock {stock}, best AIC: {best_aic_nstate} best BIC: {best_bic_nstate}")

    aic_best_model[stock]=hmm.GaussianHMM(n_components = best_aic_nstate, **param_dict).fit(insample_data)
    bic_best_model[stock]=hmm.GaussianHMM(n_components = best_bic_nstate, **param_dict).fit(insample_data)

For stock ^MERV, best AIC: 10 best BIC: 3


Model is not converging.  Current: 18922.516761095125 is not greater than 18922.640342060477. Delta is -0.1235809653517208


For stock GGAL.BA, best AIC: 10 best BIC: 3


Model is not converging.  Current: 17727.871897288493 is not greater than 17727.90753497917. Delta is -0.03563769067841349


For stock GGAL, best AIC: 10 best BIC: 3


...

Model is not converging.  Current: 17355.97899183903 is not greater than 17366.169121767216. Delta is -10.190129928185343
Model is not converging.  Current: 16823.532553851022 is not greater than 16825.761326282656. Delta is -2.2287724316338426
Model is not converging.  Current: 17293.42392956245 is not greater than 17298.170359717824. Delta is -4.746430155373673


For stock BMA, best AIC: 10 best BIC: 3


Model is not converging.  Current: 17356.143949253845 is not greater than 17360.249976997107. Delta is -4.10602774326253


For stock BBAR.BA, best AIC: 10 best BIC: 3
For stock BBAR, best AIC: 10 best BIC: 3


c:\Users\Alfred\.julia\conda\3\envs\hmm\Lib\site-packages\hmmlearn\hmm.py:340: RuntimeWarning: invalid value encountered in divide
  self.means_ = ((means_weight * means_prior + stats['obs'])
Some rows of transmat_ have zero sum because no transition from the state was ever observed.
Model is not converging.  Current: -inf is not greater than 16662.588072934486. Delta is -inf
Model is not converging.  Current: 17167.56075972157 is not greater than 17172.206482001682. Delta is -4.645722280110931


# Generating out of sample data

In [48]:
def generate_samples_residuals(model, insample_data, oos_data):
    """
    
    """
    # como el modelo es memoryless, sólo necesito 1 día de observación para saber en qué estado estoy
    # por lo tanto, en vez de complicarme con dos datasets, puedo agregarle el ultimo día de insample_data al ppio de oos_data
    # pseudocodigo
    oos_data=pd.concat([insample_data[-1:], oos_data])
    del insample_data

    samples=pd.DataFrame(columns=oos_data.columns)
    residuals=pd.DataFrame(columns=oos_data.columns)

    # for i=0
    for i in range(1,
                   len(oos_data.index)):
        prev_obs=oos_data[i-1:i]

        todays_obs = oos_data[i:i+1]
        todays_date = todays_obs.index

        state=model.decode(prev_obs)[1][-1]
        # decode()[0] is the log probability, decode()[1] is the sequence of states, [-1] is the last state
        # since we have added the last datum of insample_data to oos_data, then the 
            # TODO: revisar que tenga sentido decodear solo el ultimo día.
            # La alternativa es agregar diez días de insample al principio y usar un decode con diez dias, 
            # me quedo con el ultimo valor del array que maximiza la log-likelihood de la secuencia entera
            # pero como es memoryless, not sure if it makesense
        
        sample = model.sample(n_samples=1, random_state=random_state, currstate=state)[0]
        # sample()[0] is the array with observations of the sampled variables, sample()[1] is the value of the currstate
        sample = pd.DataFrame(data=sample, columns=oos_data.columns, index=todays_date)

        samples=pd.concat([samples, sample])   
        # sampling given state t-1
        # observar realización en t+i
        residual = todays_obs-sample
        
        residuals=pd.concat([residuals, residual])
    
    return samples, residuals

In [49]:
samples, residuals = generate_samples_residuals(aic_best_model[params["index"]], insample_data=insample_data, oos_data=insample_data)

In [50]:
residuals

,BBAR_log_rets,BBAR_gk_vol
2013-01-03,-0.018839,0.000638
2013-01-04,-0.032478,0.000313
2013-01-07,0.002437,0.000189
2013-01-08,-0.028632,0.000320
2013-01-09,-0.017018,-0.000175
...,...,...
2023-05-23,0.017173,0.002469
2023-05-24,-0.001695,0.001138
2023-05-29,-0.013097,-0.000322
2023-05-30,0.004921,0.000890


# Guardado de datos

In [27]:
with open(os.path.join(resultsroute, f"""HMM_univ_{params["tablename"]}_aic_bestmodels.pickle"""), "wb") as output_file:
    pickle.dump(aic_best_model, output_file)

with open(os.path.join(resultsroute, f"""HMM_univ_{params["tablename"]}_bic_bestmodels.pickle"""), "wb") as output_file:
    pickle.dump(bic_best_model, output_file)

Los modelos sirven los residuos NO!
https://github.com/alfsn/regime-switching-hmm/issues/27

In [28]:
with open(os.path.join(resultsroute, f"""HMM_univ_{params["tablename"]}_aic_residuals.pickle"""), "wb") as output_file:
    pickle.dump(aic_best_residuals, output_file)

with open(os.path.join(resultsroute, f"""HMM_univ_{params["tablename"]}_bic_residuals.pickle"""), "wb") as output_file:
    pickle.dump(bic_best_residuals, output_file)

In [13]:
aic_best_model

{'^MERV': GaussianHMM(n_components=10, n_iter=500, random_state=42),
 'GGAL.BA': GaussianHMM(n_components=10, n_iter=500, random_state=42),
 'GGAL': GaussianHMM(n_components=10, n_iter=500, random_state=42),
 'YPFD.BA': GaussianHMM(n_components=10, n_iter=500, random_state=42),
 'YPF': GaussianHMM(n_components=10, n_iter=500, random_state=42),
 'EDN.BA': GaussianHMM(n_components=10, n_iter=500, random_state=42),
 'EDN': GaussianHMM(n_components=10, n_iter=500, random_state=42),
 'BMA.BA': GaussianHMM(n_components=10, n_iter=500, random_state=42),
 'BMA': GaussianHMM(n_components=10, n_iter=500, random_state=42),
 'BBAR.BA': GaussianHMM(n_components=10, n_iter=500, random_state=42),
 'BBAR': GaussianHMM(n_components=10, n_iter=500, random_state=42)}

In [ ]:
def plot_close_rets_vol(model, data, key, ):
    model=models[f"{key}_{comp}_model"]
    prediction=models[f"{key}_{comp}_prediction"]
    states=set(prediction)

    fig=plt.figure(figsize = (20, 20))
    plt.tight_layout()
    plt.title(f"{key} Close, Log returns and intraday Vol\n{comp} states")

    for subplot, var in zip(range(1,4), ["Close", "log_rets", "gk_vol"]):    
        plt.subplot(3,1,subplot)
        for i in set(prediction):
            state = (prediction == i)
            x = data[key].index[state]
            y = data[key][var].iloc[state]
            plt.plot(x, y, '.')
        plt.legend(states, fontsize=16)
        
        plt.grid(True)
        plt.xlabel("datetime", fontsize=16)
        plt.ylabel(var, fontsize=16)
            
    plt.savefig(os.path.join(resultsroute, "graphs", 
                             f"HMM", 
                             f"{key}_model_{comp}.png"))

In [ ]:
for model in [aic_best_model, bic_best_model]>

## Coefficients 

In [ ]:
models.keys()

dict_keys(['^MERV_2_model', '^MERV_2_prediction', '^MERV_3_model', '^MERV_3_prediction', '^MERV_4_model', '^MERV_4_prediction', 'GGAL_2_model', 'GGAL_2_prediction', 'GGAL_3_model', 'GGAL_3_prediction', 'GGAL_4_model', 'GGAL_4_prediction', 'GGAL.BA_2_model', 'GGAL.BA_2_prediction', 'GGAL.BA_3_model', 'GGAL.BA_3_prediction', 'GGAL.BA_4_model', 'GGAL.BA_4_prediction', 'YPF_2_model', 'YPF_2_prediction', 'YPF_3_model', 'YPF_3_prediction', 'YPF_4_model', 'YPF_4_prediction', 'YPFD.BA_2_model', 'YPFD.BA_2_prediction', 'YPFD.BA_3_model', 'YPFD.BA_3_prediction', 'YPFD.BA_4_model', 'YPFD.BA_4_prediction', 'EDN_2_model', 'EDN_2_prediction', 'EDN_3_model', 'EDN_3_prediction', 'EDN_4_model', 'EDN_4_prediction', 'EDN.BA_2_model', 'EDN.BA_2_prediction', 'EDN.BA_3_model', 'EDN.BA_3_prediction', 'EDN.BA_4_model', 'EDN.BA_4_prediction', 'BMA_2_model', 'BMA_2_prediction', 'BMA_3_model', 'BMA_3_prediction', 'BMA_4_model', 'BMA_4_prediction', 'BMA.BA_2_model', 'BMA.BA_2_prediction', 'BMA.BA_3_model', 'BMA.B

In [ ]:
for model in models.keys()
    print(model)
    print("Matriz de transicion:")
    print(model.transmat_)
    print("Matriz de emisiones:")
    print(model.means_)

SyntaxError: expected ':' (2188482437.py, line 1)

## HMM Selection

Selecting the Number of States in Hidden Markov Models: Pragmatic Solutions Illustrated Using Animal Movement
https://sci-hub.st/10.1007/s13253-017-0283-8